https://github.com/bheinzerling/bpemb

In [14]:
from bpemb import BPEmb

EMB_DIM = 300
EMB_VS = 3000

bpemb = BPEmb(lang="es", dim=EMB_DIM, vs=EMB_VS)

Se guardan los embeddings en un formato visualizable en el [visualizador de embbedings de TF](http://projector.tensorflow.org/?config=http://nlp.h-its.org/bpemb/data/en/projector.config.json).

Código extraido de https://gist.github.com/BrikerMan/7bd4e4bd0a00ac9076986148afc06507.

In [15]:
import io

def export_emb(w2v):
    out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
    out_m = io.open('meta.tsv', 'w', encoding='utf-8')

    # Write meta file and vector file
    for index in range(len(w2v.index2word)):
        word = w2v.index2word[index]
        vec = w2v.vectors[index]
        out_m.write(word + "\n")
        out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_v.close()
    out_m.close()

In [17]:
print(bpemb.encode("freestyle"))

print(bpemb.most_similar("beat"))

export_emb(bpemb.emb)

['▁fre', 'est', 'y', 'le']


KeyError: "word 'beat' not in vocabulary"

Get tokens from text

In [16]:
import os,io
import unicodedata

def get_text(corpus):
    symbols = ['\n','?','¿',',','.','"',':',"'",'(',')']
    with io.open(corpus, encoding='utf-8') as f:
        text = f.read().lower().replace('\xa0', ' ').replace('-','').replace('\ufeff','')
        for s in symbols:
            text = text.replace(s,' '+s+' ')

    text = unicodedata.normalize('NFC',text)
    return text

text = get_text("./merge.txt")

tokenized_text = bpemb.encode(text)

text_tokens_ids = bpemb.encode_ids(text)

print(text[:50],'\n')
print(tokenized_text[:50],'\n')
print(text_tokens_ids[:50],'\n')

 
  
 en la improvisación 
 lamentablemente yo mue 

['▁', '\n', '▁', '\n', '▁en', '▁la', '▁im', 'pro', 'vis', 'ación', '▁', '\n', '▁la', 'm', 'enta', 'ble', 'mente', '▁yo', '▁muer', 'do', '▁como', '▁león', '▁', '\n', '▁yo', '▁te', '▁pr', 'endo', '▁fue', 'go', '▁,', '▁el', '▁concep', 'to', '▁es', '▁juego', '▁', '\n', '▁y', '▁contra', '▁m', 'í', '▁es', '▁como', '▁un', '▁parque', '▁de', '▁divers', 'ión', '▁'] 

[2905, 0, 2905, 0, 22, 20, 280, 531, 2010, 229, 2905, 0, 20, 2918, 332, 526, 181, 1592, 975, 54, 121, 2315, 2905, 0, 1592, 178, 258, 492, 146, 147, 1577, 30, 2238, 56, 58, 1716, 2905, 0, 33, 575, 24, 2931, 58, 121, 40, 2246, 5, 1281, 168, 2905] 



Filter words and get x,y

In [18]:
SEQ_LEN = 5

def shape_text(sequence_length, text, step=1):
    # cut the text in semi-redundant sequences of SEQUENCE_LEN words
    sentences = []
    next_words = []
    for i in range(0, len(text) - sequence_length, step):
        sentences.append(text[i + 1: i + sequence_length + 1])
        next_words.append(text[i])
    
    print('Total sequences:', len(sentences))
    return sentences, next_words

#def split_set(sentences, next_words)

words = set(text_tokens_ids)

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

sentences, next_words = shape_text(SEQ_LEN, text_tokens_ids)
#next_words_oh = [[1 if i == word_indices[word] else 0 for i in range(len(words))] for word in next_words]

Total sequences: 132372


Model definition

In [19]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM, Dropout, Activation


# Data generator for fit and evaluate
def generator(sentence_list, next_word_list, batch_size): 
    while True:
        for i in range(int(len(sentence_list)/batch_size)):
            x = sentence_list[i*batch_size:(i+1)*batch_size]
            x = np.array([np.array(xi) for xi in x])
            y = next_word_list[i*batch_size:(i+1)*batch_size]
            y = [[1 if i == word_indices[word] else 0 for i in range(len(words))] for word in y]
            y = np.array([np.array(yi) for yi in y])
            yield x, y
            #yield sentence_list[i*batch_size:(i+1)*batch_size], next_word_list[i*batch_size:(i+1)*batch_size]

LSTM_SIZE = 256
EMB_TRAIN = False

model = Sequential()

# define the model
model = Sequential()
model.add(Embedding(EMB_VS, EMB_DIM, weights=[bpemb.vectors], input_length=SEQ_LEN, trainable=EMB_TRAIN))
model.add(Bidirectional(LSTM(256)))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 300)            900000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               1140736   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2231)              1144503   
_________________________________________________________________
activation_3 (Activation)    (None, 2231)              0         
Total params: 3,185,239
Trainable params: 2,285,239
Non-trainable params: 900,000
_________________________________________________________________
None


Model training

In [20]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

weights_file = 'weights_embDim{}_embVS{}_embTrain{}_seqLen{}.h5_lstmSize{}'.format(EMB_DIM, EMB_VS, EMB_TRAIN, SEQ_LEN, LSTM_SIZE)

BATCH_SIZE = 256
EPOCHS = 250

if weights_file in os.listdir("./"):
        print("Weights loaded")
        model.load_weights("./" + weights_file)


model.fit(
    generator(sentences, next_words, BATCH_SIZE),
    epochs=EPOCHS,
    steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1
)

model.save_weights(weights_file)

Epoch 1/250
518/518 [==============================] - 37s 66ms/step - loss: 5.4929 - accuracy: 0.1560
Epoch 2/250
518/518 [==============================] - 34s 66ms/step - loss: 4.1915 - accuracy: 0.25911s - loss: 4.1956 - accuracy - ETA
Epoch 3/250
518/518 [==============================] - 34s 66ms/step - loss: 3.7658 - accuracy: 0.3012
Epoch 4/250
518/518 [==============================] - 34s 66ms/step - loss: 3.4800 - accuracy: 0.3307
Epoch 5/250
518/518 [==============================] - 34s 66ms/step - loss: 3.2430 - accuracy: 0.3558
Epoch 6/250
518/518 [==============================] - 34s 66ms/step - loss: 3.0355 - accuracy: 0.3801
Epoch 7/250
518/518 [==============================] - 34s 66ms/step - loss: 2.8536 - accuracy: 0.4048
Epoch 8/250
518/518 [==============================] - 34s 67ms/step - loss: 2.6845 - accuracy: 0.4264
Epoch 9/250
518/518 [==============================] - 34s 66ms/step - loss: 2.5244 - accuracy: 0.4523
Epoch 10/250
518/518 [=================

518/518 [==============================] - 35s 67ms/step - loss: 0.4021 - accuracy: 0.8853
Epoch 72/250
518/518 [==============================] - 35s 67ms/step - loss: 0.3938 - accuracy: 0.8872
Epoch 73/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3917 - accuracy: 0.8869: 33s - loss: 0.3927 - accuracy - ETA: 32s - loss: 0.3949 - accuracy: 0.8 - ETA: 32s - loss: 0.3 - ETA: 30s - - ETA: 19 - ETA: 17s - loss - ETA: 0s - loss: 0.3918 -  - ETA: 0s - loss: 0.3918 - accu
Epoch 74/250
518/518 [==============================] - 35s 67ms/step - loss: 0.3890 - accuracy: 0.8882: 34s - los - ETA: 20s - loss:  - ETA: 18s - l - ETA: 16s - loss: 0.3938 -  - ETA: 15s - loss: 0 - ETA: 10s - loss: 0.3924 - accur - ETA: 10s - lo - ETA: 7s - loss: 0.391 - ETA: 1s -
Epoch 75/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3826 - accuracy: 0.8907
Epoch 76/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3847 - accuracy: 0.8894: 32s - los

518/518 [==============================] - 34s 66ms/step - loss: 0.3181 - accuracy: 0.9073: 31s - loss: - ETA: 30s - loss: 0.3260 - accuracy:  - ETA: 29s - loss: 0.32 - ETA: - ETA: 22s - loss: 0.3213 - accuracy: -
Epoch 138/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3145 - accuracy: 0.9077
Epoch 139/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3218 - accuracy: 0.9057
Epoch 140/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3157 - accuracy: 0.90821s - loss: 0.3156 -  - E
Epoch 141/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3103 - accuracy: 0.9095A: 2s - loss: 0.3
Epoch 142/250
518/518 [==============================] - 34s 66ms/step - loss: 0.3103 - accuracy: 0.90988s - ETA: 7s - loss: 0.310 - ETA: 5s - loss: 0 - ETA: 4s - loss: 0.3101 - accuracy: 0.90 - ETA: 4s - loss: - ETA - ETA: 0s - loss: 0.310
Epoch 143/250
518/518 [==============================] - 34s 67ms/step - loss: 0.312

518/518 [==============================] - 35s 67ms/step - loss: 0.2955 - accuracy: 0.9139
Epoch 202/250
518/518 [==============================] - 35s 67ms/step - loss: 0.2901 - accuracy: 0.9150
Epoch 203/250
518/518 [==============================] - 34s 66ms/step - loss: 0.2946 - accuracy: 0.9136: 23s - loss: 0.2927 - accuracy: 0.9 - ETA: 23s - loss: 0.2928 - acc - ETA: 5s - loss: 0.2937 -  - ETA: 5s - loss: 0.2938 - ac - ETA: 4s - los - ETA: 3s - - ETA: 2s - loss: 0.2942 - accura - ETA: 2s - los -
Epoch 204/250
518/518 [==============================] - 34s 66ms/step - loss: 0.2904 - accuracy: 0.9149 - ETA: 0s - loss: 0.2904 - accura
Epoch 205/250
518/518 [==============================] - 34s 66ms/step - loss: 0.2848 - accuracy: 0.9169: 18s - loss: 0.2794 - accur - ETA: 0s - loss: 0.284
Epoch 206/250
518/518 [==============================] - 34s 66ms/step - loss: 0.2899 - accuracy: 0.9155
Epoch 207/250
518/518 [==============================] - 34s 66ms/step - loss: 0.2878 - accu

Text generation

In [ ]:
# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

seed = "pokemon"
text = np.flip(np.array(bpemb.encode_ids(seed)))

for i in range(100):
    inp = text[-SEQ_LEN:]
    
    out = model.predict(inp)[0]

    next_id = sample(out,1.0)
    #next_id = np.argmax(out)
    text = np.append(text,next_id)
    
text = np.flip(text)

text = list(map(lambda x: indices_word[x] if (x in indices_word.keys()) else int(x), text))

text = bpemb.decode_ids(text)

print(text)


Pytorch test

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.tensor as tensor
import torch.optim as optim

BATCH_SIZE = 32
EPOCHS = 1

# Data generator for fit and evaluate
def generator(sentence_list, next_word_list, batch_size): 
    for i in range(int(len(sentence_list)/batch_size)):
        yield tensor(sentence_list[i*batch_size:(i+1)*batch_size]), tensor(next_word_list[i*batch_size:(i+1)*batch_size])


class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(tensor(bpemb.vectors))
        self.lstm = nn.LSTM(EMB_DIM, 128, bidirectional=True, batch_first=True)
        self.drop = nn.Dropout()
        self.linear = nn.Linear(256, len(words))
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        
        # La LSTM devuelve una secuencia de 4, me quedo solo con el ultimo
        x = tensor([[i for i in b[-1]] for b in x])
        
        x = self.drop(x)
        x = self.linear(x)
        x = self.softmax(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)
    
net = Model().to(device)
print(net,'\n')

net.zero_grad()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters())


inp_ex = tensor(sentences[0:BATCH_SIZE]).to(device)
print('Example input size:', inp_ex.size())

out_ex = net(inp_ex)
print('Example output size:', out_ex.size())

target_ex = tensor(next_words_idx[0:BATCH_SIZE]).to(device)
print('Target output size:', target_ex.size())

loss = criterion(out_ex, target_ex)
print(loss)
'''


for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, data in enumerate(generator(sentences, next_words_idx, BATCH_SIZE), 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        net.zero_grad()
        optimizer.zero_grad()
        
        outputs = net(inputs)
        print(inputs.size(), labels.size(), outputs.size())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0


print('Finished Training')
'''

cpu
Model(
  (embedding): Embedding(100000, 100)
  (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  (drop): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=256, out_features=7921, bias=True)
  (softmax): Softmax(dim=1)
) 

Example input size: torch.Size([32, 4])
Example output size: torch.Size([32, 7921])


NameError: name 'next_words_idx' is not defined